# Partie 3 : Chargement des Données

Ce notebook présente le processus de chargement des données transformées dans notre base de données MySQL pour notre projet ETL sur les pandémies.

## Importation des bibliothèques nécessaires

In [ ]:
import pandas as pd
import numpy as np
import os
import sqlalchemy
from sqlalchemy import create_engine, text
import pymysql
import warnings

# Ignorer les avertissements
warnings.filterwarnings('ignore')

# Configuration pour afficher plus de colonnes
pd.set_option('display.max_columns', None)

## Chargement des données nettoyées

In [ ]:
# Répertoire des données nettoyées
clean_data_dir = './donnees_nettoyees/'

# Chemins des fichiers de données
confirmed_data_path = os.path.join(clean_data_dir, 'cas_confirmes_clean.csv')
deaths_data_path = os.path.join(clean_data_dir, 'deces_clean.csv')
recovered_data_path = os.path.join(clean_data_dir, 'guerisons_clean.csv')
locations_data_path = os.path.join(clean_data_dir, 'localisations_clean.csv')
pandemics_data_path = os.path.join(clean_data_dir, 'pandemies_clean.csv')

# Chargement des données
try:
    confirmed_df = pd.read_csv(confirmed_data_path)
    print(f"Données de cas confirmés chargées avec succès. Forme: {confirmed_df.shape}")
except Exception as e:
    print(f"Erreur lors du chargement des données de cas confirmés: {e}")
    confirmed_df = None

try:
    deaths_df = pd.read_csv(deaths_data_path)
    print(f"Données de décès chargées avec succès. Forme: {deaths_df.shape}")
except Exception as e:
    print(f"Erreur lors du chargement des données de décès: {e}")
    deaths_df = None
    
try:
    recovered_df = pd.read_csv(recovered_data_path)
    print(f"Données de guérisons chargées avec succès. Forme: {recovered_df.shape}")
except Exception as e:
    print(f"Erreur lors du chargement des données de guérisons: {e}")
    recovered_df = None
    
try:
    locations_df = pd.read_csv(locations_data_path)
    print(f"Données de localisations chargées avec succès. Forme: {locations_df.shape}")
except Exception as e:
    print(f"Erreur lors du chargement des données de localisations: {e}")
    locations_df = None
    
try:
    pandemics_df = pd.read_csv(pandemics_data_path)
    print(f"Données de pandémies chargées avec succès. Forme: {pandemics_df.shape}")
except Exception as e:
    print(f"Erreur lors du chargement des données de pandémies: {e}")
    pandemics_df = None

## Configuration de la connexion à la base de données

In [ ]:
# Paramètres de connexion à la base de données
db_user = 'root'
db_password = 'votre_mot_de_passe'
db_host = 'localhost'
db_port = '3306'
db_name = 'pandemies_db'

# Chaîne de connexion
connection_string = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

# Création du moteur SQLAlchemy
try:
    engine = create_engine(connection_string)
    print("Connexion à la base de données établie avec succès.")
except Exception as e:
    print(f"Erreur lors de la connexion à la base de données: {e}")
    engine = None

## Vérification de la connexion à la base de données

In [ ]:
# Vérification de la connexion
if engine is not None:
    try:
        with engine.connect() as conn:
            result = conn.execute(text("SELECT 1"))
            print("Test de connexion réussi.")
            
            # Affichage des tables existantes
            tables = engine.table_names()
            print(f"Tables existantes dans la base de données: {tables}")
    except Exception as e:
        print(f"Erreur lors du test de connexion: {e}")

## Fonctions de chargement des données

In [ ]:
def load_table(df, table_name, engine, if_exists='replace'):
    """
    Fonction pour charger un DataFrame dans une table de la base de données
    """
    if df is None or engine is None:
        print(f"Impossible de charger les données dans la table {table_name}.")
        return False
    
    try:
        # Chargement des données dans la base de données
        df.to_sql(table_name, engine, if_exists=if_exists, index=False)
        print(f"Données chargées avec succès dans la table {table_name}.")
        
        # Vérification du nombre de lignes chargées
        with engine.connect() as conn:
            result = conn.execute(text(f"SELECT COUNT(*) FROM {table_name}"))
            count = result.fetchone()[0]
            print(f"Nombre de lignes dans la table {table_name}: {count}")
        
        return True
    except Exception as e:
        print(f"Erreur lors du chargement des données dans la table {table_name}: {e}")
        return False

## Chargement des données dans la base de données

In [ ]:
# Chargement des données de pandémies
if pandemics_df is not None and engine is not None:
    load_table(pandemics_df, 'pandemie', engine)

In [ ]:
# Chargement des données de localisations
if locations_df is not None and engine is not None:
    load_table(locations_df, 'localisation', engine)

In [ ]:
# Chargement des données de cas confirmés
if confirmed_df is not None and engine is not None:
    load_table(confirmed_df, 'cas_confirmes', engine)

In [ ]:
# Chargement des données de décès
if deaths_df is not None and engine is not None:
    load_table(deaths_df, 'deces', engine)

In [ ]:
# Chargement des données de guérisons
if recovered_df is not None and engine is not None:
    load_table(recovered_df, 'guerisons', engine)

## Vérification des données chargées

In [ ]:
def query_table(query, engine):
    """
    Fonction pour exécuter une requête SQL et afficher les résultats
    """
    if engine is None:
        print("Impossible d'exécuter la requête: pas de connexion à la base de données.")
        return None
    
    try:
        # Exécution de la requête
        result = pd.read_sql(query, engine)
        return result
    except Exception as e:
        print(f"Erreur lors de l'exécution de la requête: {e}")
        return None

In [ ]:
# Vérification des données de pandémies
if engine is not None:
    result = query_table("SELECT * FROM pandemie LIMIT 5", engine)
    if result is not None:
        print("Aperçu des données de pandémies:")
        display(result)

In [ ]:
# Vérification des données de localisations
if engine is not None:
    result = query_table("SELECT * FROM localisation LIMIT 5", engine)
    if result is not None:
        print("Aperçu des données de localisations:")
        display(result)

In [ ]:
# Vérification des données de cas confirmés
if engine is not None:
    result = query_table("SELECT * FROM cas_confirmes LIMIT 5", engine)
    if result is not None:
        print("Aperçu des données de cas confirmés:")
        display(result)

## Exemples de requêtes d'analyse

In [ ]:
# Top 5 des pays avec le plus de cas confirmés
if engine is not None:
    query = """
    SELECT l.pays, MAX(c.cas_cumules) as total_cas
    FROM cas_confirmes c
    JOIN localisation l ON c.id_localisation = l.id_localisation
    GROUP BY l.pays
    ORDER BY total_cas DESC
    LIMIT 5
    """
    result = query_table(query, engine)
    if result is not None:
        print("Top 5 des pays avec le plus de cas confirmés:")
        display(result)

In [ ]:
# Top 5 des pays avec le plus de décès
if engine is not None:
    query = """
    SELECT l.pays, MAX(d.deces_cumules) as total_deces
    FROM deces d
    JOIN localisation l ON d.id_localisation = l.id_localisation
    GROUP BY l.pays
    ORDER BY total_deces DESC
    LIMIT 5
    """
    result = query_table(query, engine)
    if result is not None:
        print("Top 5 des pays avec le plus de décès:")
        display(result)

In [ ]:
# Évolution des cas par mois
if engine is not None:
    query = """
    SELECT 
        DATE_FORMAT(date, '%Y-%m') as mois,
        SUM(nouveaux_cas) as total_nouveaux_cas
    FROM cas_confirmes
    GROUP BY DATE_FORMAT(date, '%Y-%m')
    ORDER BY mois
    """
    result = query_table(query, engine)
    if result is not None:
        print("Évolution des cas par mois:")
        display(result)
        
        # Visualisation
        plt.figure(figsize=(12, 6))
        plt.bar(result['mois'], result['total_nouveaux_cas'])
        plt.title('Évolution des nouveaux cas par mois')
        plt.xlabel('Mois')
        plt.ylabel('Nouveaux cas')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

## Résumé du chargement

Dans ce notebook, nous avons :
1. Chargé les données nettoyées et transformées
2. Configuré la connexion à la base de données MySQL
3. Chargé les données dans les tables correspondantes
4. Vérifié les données chargées
5. Exécuté quelques requêtes d'analyse pour démontrer l'utilisation des données

Le processus ETL est maintenant complet : nous avons extrait les données brutes, les avons transformées et nettoyées, puis les avons chargées dans une base de données relationnelle pour analyse.